## 0. Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from ipywidgets import interactive
import matplotlib as mpl

%matplotlib widget
# %matplotlib notebook

## 1. Daten generieren

Generiere Trainings- und Testdaten mit einem Merkmal und einer kontinuierlichen Zielvariablen.
Zwischen den Variabeln herrscht ein logistischer Zusammenhang.

In [2]:
# der wahre funktionale Zusammenhang zwischen
def true_function(x):
    f = 5 / (1 + np.exp(-x + 2))
    return f

rng = np.random.RandomState(1)

# Daten - Beobachtungen / Merkmale
x_train = 10 * rng.rand(20)
x_test = 10 * rng.rand(20)

# Daten - Zielvariablen
y_train = true_function(x_train) + 0.5 * rng.randn(20)
y_test = true_function(x_test) + 0.5 * rng.randn(20)

# Zum Plotten
xx = np.linspace(0, 10)
ff = true_function(xx)

## 1.1. Visualisierung der Daten

In [10]:
plt.figure()
# plt.plot(xx, ff)
plt.scatter(x_train, y_train, alpha=0.7)
plt.scatter(x_test, y_test, alpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 1.2. Visualisierung der Verlustfunktion

In [4]:
# convenience functions and variables zur Darstellung der Verlustfunktion
def empirical_risk(w, b, x_sample, y_sample):
    # makes heavy use of broadcasting
    W = np.repeat(w[..., np.newaxis], x_sample.shape[0], axis=-1)
    B = np.repeat(b[..., np.newaxis], x_sample.shape[0], axis=-1)
    Y_pred = W * x_sample + B
    loss = np.mean((Y_pred - y_sample)**2, axis=-1)
    return loss

def weight_norm(W, B):
    return W**2 + B**2

def L1_norm(W, B):
    return np.abs(W) + np.abs(B)

ws = np.linspace(-10, 10, 1000)
bs = np.linspace(-10, 10, 1000)

def get_argmin(L):
    argmin = np.argmin(L)
    argmin = np.unravel_index(argmin, L.shape)
    
    return ws[argmin[0]], bs[argmin[1]]

W, B = np.meshgrid(ws, bs)
L = empirical_risk(W, B, x_train, y_train)
L_reg = weight_norm(W, B)
L_reg_l1 = L1_norm(W, B)


L_test = empirical_risk(W, B, x_test, y_test)

L_min, L_max = L.min(), L.max()

### 1.2.1. Empirsche Verlustfunktion ohne Regularisierung

In [5]:
from mpl_toolkits import mplot3d

fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection="3d")
ax.contour3D(W, B, L, 50, cmap="viridis")
ax.set_xlabel(r"$w$")
ax.set_ylabel(r"$b$")
ax.set_zlabel(r"$\mathcal{L}_{E}$");
ax.view_init(30, 75)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 1.2.2. Regularisierungsfunktion

In [11]:
plt.close("all")

fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection="3d")

ax.contour3D(W, B, L_reg, 50, cmap="Greens")
# ax.contour3D(W, B, lambda_*L_reg, 50, cmap="Greens")
# ax.contour3D(W, B, L_reg_l1, 50, cmap="Greens")

ax.set_xlabel(r"$w$")
ax.set_ylabel(r"$b$")
ax.set_zlabel(r"$\mathcal{L}_{reg}$");
ax.view_init(30, 75)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 1.2.3. Empirische Verlustfunktion + Regularisierungsfunktion

In [12]:
plt.close("all")

alpha = 1.0

fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection="3d")
ax.contour3D(W, B, L + alpha * L_reg, 50, cmap="rainbow")
ax.set_xlabel(r"$w$")
ax.set_ylabel(r"$b$")
ax.set_zlabel(r"$\mathcal{L}_E + \mathcal{L}_{reg}$");
ax.view_init(30, 75)

fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection="3d")

ax.contour3D(W, B, L, 50, cmap="viridis")

ax.set_xlabel(r"$w$")
ax.set_ylabel(r"$b$")
ax.set_zlabel(r"$\mathcal{L}_E$");
ax.view_init(30, 75)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 1.2.4 Empirische Verlustfunktion vs Test-Verlustfunktion

In [13]:
alpha = 5.0

plt.close("all")

fig = plt.figure(figsize=(12, 6))

plt.subplot(131)
plt.contour(W, B, L, 50, cmap="viridis")
plt.title(r"$\mathcal{L}_E$")

plt.subplot(132)
plt.contour(W, B, L + alpha*L_reg , 50, cmap="rainbow")
plt.title(r"$\mathcal{L}_E + \alpha \cdot \mathcal{L}_{reg}$")


plt.subplot(133)
plt.contour(W, B, L_test, 50, cmap="seismic")
plt.title(r"$\mathcal{L}_T$");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …